In [1]:
# Source: https://github.com/unit8co/darts/blob/master/examples/01-multi-time-series-and-covariates.ipynb
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
)
from darts.metrics import mape, smape
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.datasets import AirPassengersDataset, MonthlyMilkDataset

# for reproducibility
torch.manual_seed(1)
np.random.seed(1)

2022-06-09 20:37:31 prophet.plot ERROR: Importing plotly failed. Interactive plots will not work.


In [3]:
# Encode data for the model
training_data = pd.read_csv('./data/training_data.csv')

# Temp df to test function
data_unseen = training_data[2390:2629]
data_unseen.columns = data_unseen.columns.str.lower()
training_data = training_data[0:2390]
training_data.columns = training_data.columns.str.lower()
training_data = training_data.drop(columns=['name', 'last_sale','sales','presale'])
training_data.head(3)
list(training_data.columns)

/var/folders/pr/s8vz2jrj0gzcy0cb3dtmqx800000gn/T/ipykernel_63171/1019096430.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  training_data = pd.read_csv('./data/training_data.csv')


['timestamp',
 'token_id',
 'owner',
 'mouth',
 'eyes',
 'hat',
 'fur',
 'background',
 'clothes',
 'earring',
 'eth_fees',
 'eth_sale price',
 'usd_platform_fees',
 'usd_sale_price',
 'index',
 'average_price',
 'max_price',
 'min_price',
 'total_sales',
 'total_volume',
 'unique_buyers',
 'unique_sellers',
 'buyers_ratio',
 'btc_close',
 'btc_volume',
 'eth_close',
 'eth_volume',
 'gas_price',
 'nft',
 'nfts',
 'opensea']

In [ ]:
# Test out put -> can be deleted later
training_data_temp = training_data

In [ ]:
col_to_encode = training_data[['owner', 'mouth', 'eyes', 'hat', 'fur', 'background', 'clothes', 'earring']]
col_to_encode.fillna('Other', inplace=True)

In [ ]:
# OneHotEncode data
X = col_to_encode
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc.fit(X)
print('done')

In [ ]:
col_names = ['owner', 'mouth', 'eyes', 'hat', 'fur', 'background', 'clothes', 'earring']
colnames = enc.get_feature_names(col_names)
encoded_traning_data =  pd.DataFrame(enc.fit_transform(X), columns = colnames)
print('done')

In [ ]:
encoded_traning_data.head(3)

In [ ]:
training_data_temp = training_data_temp.drop(columns=col_names)
training_data_dic = training_data_temp.to_dict('list')
encoded_traning_data_dic = encoded_traning_data.to_dict('list')

In [ ]:
def merge(d1, d2):
    result = {**d1, **d2}
    return result
encoded_traning_data_temp = merge(encoded_traning_data_dic, training_data_dic)
encoded_traning_data_temp = pd.DataFrame.from_dict(encoded_traning_data_temp)

In [ ]:
# get target index no
#list(encoded_traning_data_temp.columns)
index_no = encoded_traning_data_temp.columns.get_loc("eth_sale price")
index_no

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#sc = MinMaxScaler(feature_range=(0,1))
#encoded_traning_data_temp = encoded_traning_data_temp.drop(['timestamp'], 1)
#training_set_scaled = sc.fit_transform(encoded_traning_data_temp)
#training_set_scaled

In [2]:
# build year and month series:
air_year = datetime_attribute_timeseries(series_air_scaled, attribute="year")
air_month = datetime_attribute_timeseries(series_air_scaled, attribute="month")

milk_year = datetime_attribute_timeseries(series_milk_scaled, attribute="year")
milk_month = datetime_attribute_timeseries(series_milk_scaled, attribute="month")

# stack year and month to obtain series of 2 dimensions (year and month):
air_covariates = air_year.stack(air_month)
milk_covariates = milk_year.stack(milk_month)

# scale them between 0 and 1:
scaler_dt_air = Scaler()
air_covariates = scaler_dt_air.fit_transform(air_covariates)

scaler_dt_milk = Scaler()
milk_covariates = scaler_dt_milk.fit_transform(milk_covariates)

# split in train/validation sets:
air_train_covariates, air_val_covariates = air_covariates[:-36], air_covariates[-36:]
milk_train_covariates, milk_val_covariates = (
    milk_covariates[:-36],
    milk_covariates[-36:],
)

# plot the covariates:
plt.figure()
air_covariates.plot()
plt.title("Air traffic covariates (year and month)")

plt.figure()
milk_covariates.plot()
plt.title("Milk production covariates (year and month)")

NameError: name 'series_air_scaled' is not defined

In [ ]:
#model = keras.Sequential()
#model.add(layers.GRU(64, input_shape=(5, 5040)))
#model.add(layers.BatchNormalization())
#model.add(layers.Dense(10))
#print(model.summary())